# TODO

- улучшить парсер (убрать пустые тексты)
- убрать наны из текстов!!!!
- выкинуть некириллицу
- возьмем обученные эмбеддинги у Гали
- tf-idf

## Метрики
длина текста, tf-idf, лемматизированные тексты, максимальное и минимальное косинусное расстояние между парами слов (соседних и не соседних, если успеем посчитать), tf-idf взвешенное (умножать каждое слово на tf-idf этого слова для этого документа)

In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib
%matplotlib inline
import random
import warnings

df = pd.read_csv('data.csv', index_col='id')
df = df.sample(frac=1)
df.head()

,text,group
id,,
2699,ДОХЛЫЙ КУДАХ И ДВА НАРКОМАНА\n\nброви кудаха с...,kudah
289,"Вслед за деревянным небоскрёбом Сутягина, кану...",podkoren
1603,Возьмите скотч и попробуйте им склеить много р...,telegony
1373,Красота...,telegony
1741,"Доброту не купишь на базаре,\nИскренность у пе...",telegony


In [191]:
import warnings
warnings.filterwarnings('ignore')

In [192]:
df.describe(include=[np.object])

,text,group
count,2447,3000
unique,2429,5
top,Доброе утро!,ufo
freq,5,600


In [193]:
import nltk
import re
from pymystem3 import Mystem

nltk.download('punkt')
mystem_analyzer = Mystem()

lemmatized = []
unlemmatized = []

for doc in df['text']:
    text = re.sub("<br>", "", str(doc))
    word_list = nltk.word_tokenize(text)
    unlemmatized.append(word_list)
    lemmatized.append([mystem_analyzer.lemmatize(word)[0] for word in word_list])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [194]:
from itertools import chain

count_unlem = 0

for el in unlemmatized:
    count_unlem += len(el)

cont_lemmatized = list(chain.from_iterable(lemmatized))

print("Количество токенов в датасете:", count_unlem)
print("Количество уникальных лемм:", len(set(cont_lemmatized)))

Количество токенов в датасете: 692543
Количество уникальных лемм: 37248


In [195]:
words_per_doc = np.mean([len(text) for text in unlemmatized])
symbols_per_doc = np.mean([len(str(doc)) for doc in df["text"]])

print("Среднее количество слов на документ:", int(words_per_doc))
print("Среднее количество символов на документ:", int(symbols_per_doc))

Среднее количество слов на документ: 230
Среднее количество символов на документ: 1281


In [196]:
from nltk.corpus import stopwords
import pymorphy2 as pm2

pmm = pm2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")

# сюда добавить парсинг получше
def parse(text):
    text = re.sub(r'[^\w\s]','', str(text))
    text = [pmm.normal_forms(x)[0] for x in text.split() if x not in russian_stopwords]
    if text == [np.nan]:
        return(['nan'])
    return text

df['text'] = df['text'].apply(parse)
df["text"]

id
2699    [дохлый, куд, и, два, наркоман, бровь, кудах, ...
289     [вслед, деревянный, небоскрёб, сутягин, кануть...
1603    [взять, скотч, попробовать, склеить, поверхнос...
1373                                            [красота]
1741    [доброта, купить, базар, искренность, песнь, з...
1916                                                [nan]
2278             [гонка, нло, снять, видеокамера, канада]
1859                                           [что, это]
2658    [кудах, от, я, не, дется, заебало, спать, сто,...
2436    [куд, и, победобесие, куд, часто, дерётсйя, ко...
2565    [куд, vs, партизан, 1942, год, дед, который, п...
2477    [куд, непрескоствавенный, день, пожарка, кудах...
1798    [пусть, жизнь, шанс, счастие, повод, улыбка, в...
615                                                 [nan]
713                                                 [nan]
2207    [ярославль, сегодня, снимка, вспышка, что, так...
882     [тайна, одновременный, рождение, кришна, вринд...
2945    [ку

In [197]:
# for idx, row in df.iterrows():
#     if row['text'] == []:
#         df = df.drop(index=idx)
df

,text,group
id,,
2699,"[дохлый, куд, и, два, наркоман, бровь, кудах, ...",kudah
289,"[вслед, деревянный, небоскрёб, сутягин, кануть...",podkoren
1603,"[взять, скотч, попробовать, склеить, поверхнос...",telegony
1373,[красота],telegony
1741,"[доброта, купить, базар, искренность, песнь, з...",telegony
1916,[nan],ufo
2278,"[гонка, нло, снять, видеокамера, канада]",ufo
1859,"[что, это]",ufo
2658,"[кудах, от, я, не, дется, заебало, спать, сто,...",kudah


In [198]:
df = df.reindex(index=range(len(df)))
cross_val_folds = []

fold = []

for idx, row in df.iterrows():
    fold.append(row)
    if idx % (len(df) // 10) == 0:
        cross_val_folds.append(fold)
        fold = []

len(cross_val_folds)

10

In [199]:
cross_val_folds = cross_val_folds[1:]

for fold in cross_val_folds:
    print(len(fold))

300
300
300
300
300
300
300
300
300


## TF-IDF

In [200]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

dct = Dictionary()
#for text in lemmatized_corpus:
dct.add_documents(df["text"])
corpus = []
for idx, row in df.iterrows():
    corpus.append(dct.doc2bow(row['text']))
idfs = {}
tf_idf = TfidfModel(corpus)
for key in dct:
    try:
        idfs[dct[key]] = tf_idf.idfs[key]
    except KeyError:
        pass

In [201]:
vector = tf_idf[corpus[0][:10]]
vector

[(0, 0.36221007863501486),
 (1, 0.1467250140965477),
 (2, 0.41541282888806597),
 (3, 0.42624121900071205),
 (4, 0.26131078311247774),
 (5, 0.3324372938635929),
 (6, 0.35322561348812204),
 (7, 0.2531146050432289),
 (8, 0.24608396493680115),
 (9, 0.2546285513490461)]

## Embeddings

In [207]:
from gensim.models import Word2Vec

model = Word2Vec.load("full.model")